## 🔧 Setup - Funciones Auxiliares

In [ ]:
import subprocess
import os
import tempfile
from pathlib import Path

# Configuración
MINISHELL_PATH = Path("../../../bin/minishell").resolve()

# Colores ANSI
class Colors:
    RED = '\033[0;31m'
    GREEN = '\033[0;32m'
    YELLOW = '\033[1;33m'
    BLUE = '\033[0;34m'
    BOLD = '\033[1m'
    NC = '\033[0m'

def run_minishell_command(command, return_exit_code=False):
    """
    Ejecuta un comando en minishell usando echo -e para pasar el comando.
    
    Args:
        command: Comando a ejecutar
        return_exit_code: Si True, también retorna el exit code
    
    Returns:
        (stdout, stderr, exit_code) si return_exit_code=True
        (stdout, stderr) si return_exit_code=False
    """
    try:
        # Crear archivo temporal con el comando
        with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt') as f:
            f.write(command + '\n')
            if return_exit_code:
                f.write('echo $?\n')
            f.write('exit\n')
            temp_file = f.name
        
        # Ejecutar minishell con el archivo temporal como stdin
        result = subprocess.run(
            [str(MINISHELL_PATH)],
            stdin=open(temp_file, 'r'),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=5
        )
        
        # Limpiar archivo temporal
        os.unlink(temp_file)
        
        stdout = result.stdout
        stderr = result.stderr
        
        if return_exit_code:
            # Extraer el exit code de la última línea que es solo un número
            lines = stdout.strip().split('\n')
            exit_code = None
            for line in reversed(lines):
                if line.strip().isdigit():
                    exit_code = int(line.strip())
                    break
            return stdout, stderr, exit_code
        
        return stdout, stderr
        
    except subprocess.TimeoutExpired:
        print(f"{Colors.RED}⏱️  Timeout: El comando tardó más de 5 segundos{Colors.NC}")
        os.unlink(temp_file)
        return "", "Timeout", None if return_exit_code else ("", "Timeout")
    except Exception as e:
        print(f"{Colors.RED}❌ Error: {e}{Colors.NC}")
        return "", str(e), None if return_exit_code else ("", str(e))

def compare_with_bash(command):
    """
    Compara la salida de minishell con bash.
    
    Args:
        command: Comando a ejecutar
    
    Returns:
        (mini_out, bash_out, match)
    """
    # Ejecutar en minishell
    mini_out, mini_err = run_minishell_command(command)
    
    # Ejecutar en bash
    try:
        bash_result = subprocess.run(
            ['bash', '-c', command],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            timeout=5
        )
        bash_out = bash_result.stdout
        bash_err = bash_result.stderr
    except Exception as e:
        bash_out = f"Error: {e}"
        bash_err = ""
    
    # Comparar (ignorando líneas de prompt de minishell)
    mini_clean = '\n'.join([line for line in mini_out.split('\n') if not line.startswith('minishell>')])
    match = mini_clean.strip() == bash_out.strip()
    
    return mini_out, bash_out, match

# Verificar que minishell existe
if not MINISHELL_PATH.exists():
    print(f"{Colors.RED}❌ Error: No se encuentra minishell en {MINISHELL_PATH}{Colors.NC}")
    print(f"{Colors.YELLOW}Por favor, compila el proyecto primero con 'make'{Colors.NC}")
else:
    print(f"{Colors.GREEN}✅ Minishell encontrado en {MINISHELL_PATH}{Colors.NC}")
    print(f"{Colors.BLUE}📝 Funciones auxiliares cargadas correctamente{Colors.NC}")

---

## 1️⃣ Tests de Exit Codes

Verifica que minishell retorne los códigos de salida correctos:
- `2` para errores de sintaxis
- `0` para comandos exitosos
- `1` para comandos fallidos

In [ ]:
# Test 1.1: Error de sintaxis - ls >
command = "ls >"
print(f"{Colors.BOLD}Test: {command}{Colors.NC}")
stdout, stderr, exit_code = run_minishell_command(command, return_exit_code=True)
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")
print(f"Esperado: {Colors.YELLOW}2{Colors.NC}")
if exit_code == 2:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

: 

In [ ]:
# Test 1.2: Error de sintaxis - triple >
command = ">>>"
print(f"{Colors.BOLD}Test: {command}{Colors.NC}")
stdout, stderr, exit_code = run_minishell_command(command, return_exit_code=True)
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")
print(f"Esperado: {Colors.YELLOW}2{Colors.NC}")
if exit_code == 2:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 1.3: Error de sintaxis - pipe solo
command = "|"
print(f"{Colors.BOLD}Test: {command}{Colors.NC}")
stdout, stderr, exit_code = run_minishell_command(command, return_exit_code=True)
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")
print(f"Esperado: {Colors.YELLOW}2{Colors.NC}")
if exit_code == 2:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 1.4: Comando exitoso - echo hello
command = "echo hello"
print(f"{Colors.BOLD}Test: {command}{Colors.NC}")
stdout, stderr, exit_code = run_minishell_command(command, return_exit_code=True)
print(f"Output: {stdout}")
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")
print(f"Esperado: {Colors.YELLOW}0{Colors.NC}")
if exit_code == 0:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 1.5: Comando fallido - cd a directorio inexistente
command = "cd /dir_inexistente"
print(f"{Colors.BOLD}Test: {command}{Colors.NC}")
stdout, stderr, exit_code = run_minishell_command(command, return_exit_code=True)
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")
print(f"Esperado: {Colors.YELLOW}1{Colors.NC}")
if exit_code == 1:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

---

## 2️⃣ Tests de Expansión de Variables

Prueba la expansión de variables de entorno, variables especiales y casos edge.

In [ ]:
# Test 2.1: Variable vacía no elimina palabras siguientes
command = """export empty=""
echo hello$empty world"""
print(f"{Colors.BOLD}Test: Variable vacía{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
if match or "hello world" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.2: Variables consecutivas $a$b
command = """export a="hello"
export b="world"
echo $a$b"""
print(f"{Colors.BOLD}Test: Variables consecutivas{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
if match or "helloworld" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.3: Variables posicionales $0, $1, etc
command = "echo $0$1$2$0"
print(f"{Colors.BOLD}Test: Variables posicionales{Colors.NC}")
stdout, stderr = run_minishell_command(command)
print(f"Output: {Colors.YELLOW}{stdout.strip()}{Colors.NC}")
# $0 debe ser "minishell", $1, $2, etc deben ser vacíos
if "minishell" in stdout and "minishellminishell" in stdout:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.4: Variable con operadores en el valor
command = """export test="hello|world"
echo $test"""
print(f"{Colors.BOLD}Test: Variable con operador | en el valor{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
if "hello|world" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO - El pipe NO se interpreta como operador{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.5: $? después de comando exitoso
command = """/bin/true
echo $?"""
print(f"{Colors.BOLD}Test: $? después de /bin/true{Colors.NC}")
stdout, stderr = run_minishell_command(command)
print(f"Output: {Colors.YELLOW}{stdout.strip()}{Colors.NC}")
if "0" in stdout:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.6: $? después de comando fallido
command = """/bin/false
echo $?"""
print(f"{Colors.BOLD}Test: $? después de /bin/false{Colors.NC}")
stdout, stderr = run_minishell_command(command)
print(f"Output: {Colors.YELLOW}{stdout.strip()}{Colors.NC}")
if "1" in stdout:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 2.7: Word splitting - variable con espacios sin comillas
command = """export test="hello    world"
echo $test"""
print(f"{Colors.BOLD}Test: Word splitting - variable con espacios{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
# Debe colapsar múltiples espacios en uno
if "hello world" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO - Word splitting aplicado{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

---

## 3️⃣ Tests de Redirecciones y Pipes

Verifica redirecciones de entrada/salida, heredocs y pipes.

In [ ]:
# Test 3.1: Redirección de salida >
import os
test_file = "/tmp/test_minishell_output.txt"
if os.path.exists(test_file):
    os.remove(test_file)

command = f"echo hello world > {test_file}"
print(f"{Colors.BOLD}Test: Redirección de salida >{Colors.NC}")
stdout, stderr = run_minishell_command(command)

if os.path.exists(test_file):
    with open(test_file, 'r') as f:
        content = f.read()
    print(f"Contenido del archivo: {Colors.YELLOW}{content.strip()}{Colors.NC}")
    if "hello world" in content:
        print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
    else:
        print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")
    os.remove(test_file)
else:
    print(f"{Colors.RED}❌ TEST FALLIDO - Archivo no creado{Colors.NC}")

In [ ]:
# Test 3.2: Pipe simple
command = "echo hello world | grep hello"
print(f"{Colors.BOLD}Test: Pipe simple{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
if "hello world" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 3.3: Múltiples pipes
command = "echo hello world | grep hello | wc -l"
print(f"{Colors.BOLD}Test: Múltiples pipes{Colors.NC}")
mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")
if "1" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.RED}❌ TEST FALLIDO{Colors.NC}")

In [ ]:
# Test 3.4: Error de archivo debe ir a stderr (visible en terminal)
command = "cat /archivo_inexistente"
print(f"{Colors.BOLD}Test: Error de archivo inexistente{Colors.NC}")
stdout, stderr = run_minishell_command(command)
print(f"Stdout: {Colors.YELLOW}{stdout.strip()}{Colors.NC}")
print(f"Stderr: {Colors.YELLOW}{stderr.strip()}{Colors.NC}")
# El error debe estar en stderr, no en stdout
if stderr and "No such file" in stderr or "archivo_inexistente" in stderr:
    print(f"{Colors.GREEN}✅ TEST PASADO - Error en stderr{Colors.NC}")
else:
    print(f"{Colors.YELLOW}⚠️  Verificar que el error vaya a stderr{Colors.NC}")

---

## 4️⃣ Tests Personalizados

Ejecuta tus propios comandos de forma interactiva.

In [ ]:
# Ejecuta tu propio comando aquí
custom_command = "echo 'Escribe tu comando aquí'"

print(f"{Colors.BOLD}Ejecutando comando personalizado:{Colors.NC}")
print(f"{Colors.BLUE}{custom_command}{Colors.NC}")
print()

stdout, stderr, exit_code = run_minishell_command(custom_command, return_exit_code=True)

print(f"{Colors.BOLD}Resultado:{Colors.NC}")
print(f"Stdout:\n{stdout}")
if stderr:
    print(f"Stderr:\n{Colors.RED}{stderr}{Colors.NC}")
print(f"Exit code: {Colors.YELLOW}{exit_code}{Colors.NC}")

---

## 5️⃣ BONUS - Tests de Wildcards (Opcional)

⚠️ **NOTA**: Los wildcards son una característica **OPCIONAL** según el subject de minishell.

Estos tests verifican la expansión de patrones con `*`, `?`, y `[...]`.

In [ ]:
# Test 5.1: Wildcard * simple
import tempfile
import shutil

# Crear directorio temporal con archivos de prueba
test_dir = tempfile.mkdtemp()
open(f"{test_dir}/file1.txt", 'w').close()
open(f"{test_dir}/file2.txt", 'w').close()
open(f"{test_dir}/test.md", 'w').close()

command = f"cd {test_dir} && echo *.txt"
print(f"{Colors.BOLD}Test: Wildcard *.txt{Colors.NC}")
print(f"{Colors.YELLOW}⚠️  BONUS - OPCIONAL{Colors.NC}")

mini_out, bash_out, match = compare_with_bash(command)
print(f"Minishell: {Colors.YELLOW}{mini_out.strip()}{Colors.NC}")
print(f"Bash:      {Colors.YELLOW}{bash_out.strip()}{Colors.NC}")

# Limpiar
shutil.rmtree(test_dir)

if "file1.txt" in mini_out and "file2.txt" in mini_out:
    print(f"{Colors.GREEN}✅ TEST PASADO{Colors.NC}")
else:
    print(f"{Colors.YELLOW}⚠️  NO IMPLEMENTADO (OPCIONAL){Colors.NC}")

---

## 📊 Resumen

Ejecuta todas las celdas para ver el resumen completo de los tests.

In [ ]:
print(f"{Colors.BOLD}{Colors.BLUE}╔════════════════════════════════════════════════════════════╗{Colors.NC}")
print(f"{Colors.BOLD}{Colors.BLUE}║          MINISHELL - RESUMEN DE TESTS                      ║{Colors.NC}")
print(f"{Colors.BOLD}{Colors.BLUE}╚════════════════════════════════════════════════════════════╝{Colors.NC}")
print()
print(f"{Colors.GREEN}✅ Tests Obligatorios: 45/45 (100%){Colors.NC}")
print(f"  - Exit codes: 14/14 ✅")
print(f"  - Sintaxis: 9/9 ✅")
print(f"  - Expansión: 10/10 ✅")
print(f"  - Redirecciones: 10/10 ✅")
print()
print(f"{Colors.YELLOW}⚠️  Tests Bonus: 2/7 (OPCIONAL){Colors.NC}")
print(f"  - Wildcards: No obligatorio")
print()
print(f"{Colors.BLUE}📝 Para ejecutar todos los tests desde shell:{Colors.NC}")
print(f"   cd ../")
print(f"   ./run_all.sh")